# Fama French Regression in Python

#### r - rf = alpha + beta (return_market - rf) + s_iSMB + h_iHML + error term

In [17]:
%pip install yfinance
%pip install statsmodels
# size and value effect

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [18]:
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [19]:
end = dt.date(2025, 6,30)
start = dt.date(end.year - 5, end.month, end.day)

In [20]:
#Fidelity Growth Company Fund - highly diversified (433 holdings)
funds = ['FDGRX']

In [21]:
df = yf.download(funds, start,end, auto_adjust=False)['Adj Close']
df

# We need daily returns 
df = yf.download(funds, start,end, auto_adjust=False)['Adj Close'].pct_change()
fundsret = df
fundsret = fundsret.dropna(axis=0)
fundsret

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Ticker,FDGRX
Date,
2020-07-01,0.013601
2020-07-02,0.004845
2020-07-06,0.019288
2020-07-07,-0.003275
2020-07-08,0.016429
...,...
2025-06-23,0.001994
2025-06-24,0.016418
2025-06-25,0.005139


In [54]:
#Monthly returns with lamda aggregation: cumulative return calculation
fundsret_mtl = fundsret.resample('M').agg(lambda x: (x+1).prod() - 1)
fundsret_mtl

Ticker,FDGRX
Date,
2020-07-31,0.081224
2020-08-31,0.126135
2020-09-30,-0.033199
2020-10-31,-0.028562
2020-11-30,0.138751
2020-12-31,0.037423
2021-01-31,0.013995
2021-02-28,0.020702
2021-03-31,-0.010288


In [23]:
%pip install pandas_datareader
import pandas_datareader.data as reader
import pandas as pd

Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels
Note: you may need to restart the kernel to use updated packages.


In [49]:
factors = reader.DataReader('F-F_Research_Data_Factors', 'famafrench', start, end)[0]
factors

/home/8e6871ea-9e76-4029-849d-302a75dfa8e7/.local/lib/python3.11/site-packages/pandas_datareader/famafrench.py:114: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = read_csv(StringIO("Date" + src[start:]), **params)
/home/8e6871ea-9e76-4029-849d-302a75dfa8e7/.local/lib/python3.11/site-packages/pandas_datareader/famafrench.py:114: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = read_csv(StringIO("Date" + src[start:]), **params)


,Mkt-RF,SMB,HML,RF
Date,,,,
2020-06,2.45,2.67,-2.17,0.01
2020-07,5.83,-2.35,-1.44,0.01
2020-08,7.62,-0.20,-3.02,0.01
2020-09,-3.64,0.06,-2.72,0.01
2020-10,-2.08,4.34,4.25,0.01
2020-11,12.45,5.81,2.09,0.01
2020-12,4.63,5.01,-1.68,0.01
2021-01,-0.07,7.14,3.22,0.00
2021-02,2.81,2.02,7.20,0.00


In [51]:
#Deleting the first row
factors = factors[1:]
factors

,Mkt-RF,SMB,HML,RF
Date,,,,
2020-07,5.83,-2.35,-1.44,0.01
2020-08,7.62,-0.20,-3.02,0.01
2020-09,-3.64,0.06,-2.72,0.01
2020-10,-2.08,4.34,4.25,0.01
2020-11,12.45,5.81,2.09,0.01
2020-12,4.63,5.01,-1.68,0.01
2021-01,-0.07,7.14,3.22,0.00
2021-02,2.81,2.02,7.20,0.00
2021-03,3.16,-2.33,7.35,0.00


In [55]:
#Aligning the data set so that they both have the same length
fundsret_mtl = fundsret_mtl.iloc[:-1]
fundsret_mtl.shape

(59, 1)

In [52]:
factors.shape

(59, 4)

In [57]:
#because the dates are the same in both data frames although not the same format, we can still merge them, but first transform the data format for fundsmrl\_ret
fundsret_mtl.index = factors.index
fundsret_mtl

Ticker,FDGRX
Date,
2020-07,0.081224
2020-08,0.126135
2020-09,-0.033199
2020-10,-0.028562
2020-11,0.138751
2020-12,0.037423
2021-01,0.013995
2021-02,0.020702
2021-03,-0.010288


In [68]:
merge = pd.merge(fundsret_mtl, factors, on = 'Date')
merge
#important to know that FDGRX is decimal, everything else is in %

,FDGRX,Mkt-RF,SMB,HML,RF
Date,,,,,
2020-07,0.081224,5.83,-2.35,-1.44,0.01
2020-08,0.126135,7.62,-0.20,-3.02,0.01
2020-09,-0.033199,-3.64,0.06,-2.72,0.01
2020-10,-0.028562,-2.08,4.34,4.25,0.01
2020-11,0.138751,12.45,5.81,2.09,0.01
2020-12,0.037423,4.63,5.01,-1.68,0.01
2021-01,0.013995,-0.07,7.14,3.22,0.00
2021-02,0.020702,2.81,2.02,7.20,0.00
2021-03,-0.010288,3.16,-2.33,7.35,0.00


In [76]:
#resigning and dividing 
merge[['Mkt-RF', 'SMB', 'HML', 'RF']] = merge[['Mkt-RF', 'SMB', 'HML', 'RF']]/100
merge

,FDGRX,Mkt-RF,SMB,HML,RF,FDGRX-RF
Date,,,,,,
2020-07,0.081224,0.0583,-0.0235,-0.0144,0.0001,0.071224
2020-08,0.126135,0.0762,-0.0020,-0.0302,0.0001,0.116135
2020-09,-0.033199,-0.0364,0.0006,-0.0272,0.0001,-0.043199
2020-10,-0.028562,-0.0208,0.0434,0.0425,0.0001,-0.038562
2020-11,0.138751,0.1245,0.0581,0.0209,0.0001,0.128751
2020-12,0.037423,0.0463,0.0501,-0.0168,0.0001,0.027423
2021-01,0.013995,-0.0007,0.0714,0.0322,0.0000,0.013995
2021-02,0.020702,0.0281,0.0202,0.0720,0.0000,0.020702
2021-03,-0.010288,0.0316,-0.0233,0.0735,0.0000,-0.010288


In [77]:
# creating the dependent variable for the regression r_i - r_f
merge['FDGRX-RF'] = merge.FDGRX - merge.RF

In [78]:
merge

,FDGRX,Mkt-RF,SMB,HML,RF,FDGRX-RF
Date,,,,,,
2020-07,0.081224,0.0583,-0.0235,-0.0144,0.0001,0.081124
2020-08,0.126135,0.0762,-0.0020,-0.0302,0.0001,0.126035
2020-09,-0.033199,-0.0364,0.0006,-0.0272,0.0001,-0.033299
2020-10,-0.028562,-0.0208,0.0434,0.0425,0.0001,-0.028662
2020-11,0.138751,0.1245,0.0581,0.0209,0.0001,0.138651
2020-12,0.037423,0.0463,0.0501,-0.0168,0.0001,0.037323
2021-01,0.013995,-0.0007,0.0714,0.0322,0.0000,0.013995
2021-02,0.020702,0.0281,0.0202,0.0720,0.0000,0.020702
2021-03,-0.010288,0.0316,-0.0233,0.0735,0.0000,-0.010288


In [79]:
y = merge['FDGRX-RF']
X = merge[['Mkt-RF', 'SMB', 'HML']]
#adding the const
X_sm = sm.add_constant(X)

In [80]:
model = sm.OLS(y, X_sm)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               FDGRX-RF   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.934
Method:                 Least Squares   F-statistic:                     276.1
Date:                Thu, 24 Jul 2025   Prob (F-statistic):           4.06e-33
Time:                        07:49:11   Log-Likelihood:                 161.20
No. Observations:                  59   AIC:                            -314.4
Df Residuals:                      55   BIC:                            -306.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0032      0.002      1.438      0.156      -0.001       0.008
Mkt-RF         1.1409      0.046     24.551      0.000       1.048       1.234
SMB            0.0628      0.071      0.888      0.378      -0.079       0.204
HML           -0.4310      0.051     -8.491      0.000      -0.533      -0.329
==============================================================================
Omnibus:                       12.309   Durbin-Watson:                   2.596
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               14.581
Skew:                          -0.844   Prob(JB):                     0.000682
Kurtosis:                       4.755   Cond. No.                         34.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# R squared is nice, almost 1
## Market risk ~ beta, 1.14 , fidelity fund has more market risk than the AVG asset
## Size factor slightly more positive, tend to invest rather in smaller stocks 
## Large negative loading, fidelity growth fund overweighting growth stocks relative to the whole market. 